# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Assignment 3: Parallel Programming with MPI

## Learning Objectives

At the end of the experiment, you will be able to:


* implement standard message-passing algorithms in MPI
* debug simple MPI code
* understand the basics of point-to-point communication
* understand the difference between blocking and non-blocking communication
* understand how non-blocking communication can improve program performance
* understand the basics of collective communication
* learn about the different types of collective communication


## Information


### MPI in a Nutshell

MPI stands for "Message Passing Interface". It is a library of functions (in C / Python) or subroutines (in Fortran) that you insert into source code to perform data communication between processes. MPI was developed over two years of discussions led by the MPI Forum, a group of roughly sixty people representing some forty organizations.

To know more about MPI click [here](https://computing.llnl.gov/tutorials/mpi/#What)

### Setup Steps:

In [1]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2417833" #@param {type:"string"}

In [2]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "9840913286" #@param {type:"string"}

In [3]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "M5_AST_03_MPI_C" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://learn-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


**Note:** We will be using the MPI for Python package mpi4py, Refer the [link](https://mpi4py.readthedocs.io/)

In [4]:
# Install the mpi4py package
!pip -qq install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


### Blocking Communication

In Blocking communication the process sending a message will be waiting until the process receiving has finished receiving all the information.

To know more about Blocking Communication click [here](https://stackoverflow.com/questions/10017301/mpi-blocking-vs-non-blocking#:~:text=BLOCKING%20COMMUNICATION%20%3A%20Blocking%20doesn't,buffer%20is%20available%20for%20reuse.)

#### Creating a Communicator and Getting rank of each process

- A Communicator is a collection of MPI processes that can send and receive messages to and from each other.
- A size is a number of processes in a communicator
- A Rank is a unique identifier for each process in the communicator

To know more about Communicator, size and rank click [here](https://www.codingame.com/playgrounds/349/introduction-to-mpi/mpi_comm_world-size-and-ranks)

We will be using magic command **%%writefile** to write the contents of the cell to a file.

To know more about magic commands click [here](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

In [9]:
import os
os.getcwd()

'/content'

In [2]:
from mpi4py import MPI

In [3]:
%%writefile rank.py
from mpi4py import MPI # Importing mpi4py package from MPI module
# Define a function
def main():
    # creating the communicator
    comm = MPI.COMM_WORLD
    # number of the process running the code i.e rank
    rank = comm.Get_rank()
    # total number of processes running i.e size
    size = comm.Get_size()
    # Displaying the rank and size of a communicator
    print("rank is {} and size is {}".format(rank,size))

# invoke the function
main()

Writing rank.py


To run an MPI code, we commonly use a "wrapper" called **mpirun**. Now let us see how we can use the **mpirun** program to execute the above python code using 4 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

To know more about the mpirun click [here](https://docs.oracle.com/cd/E19356-01/820-3176-10/ExecutingPrograms.html#50413574_62903)

In [14]:
!mpirun --allow-run-as-root --oversubscribe -np 4 python rank.py

rank is 1 and size is 4
rank is 2 and size is 4
rank is 3 and size is 4
rank is 0 and size is 4


We can execute the above code using wrapper `mpiexec` instead of wrapper mpirun or as a alternative to it.

Hint: [mpiexec](https://www.mpich.org/static/docs/v3.1/www1/mpiexec.html)

#### Exercise

- Run the above code, using different number of processes.

#### Point-to-point communication

The elementary communication operation in MPI is "point-to-point" communication, that is, direct communication between two processors, one of which sends and the other receives.

![Image](https://cdn.iisc.talentsprint.com/CDS/Images/pointopoint.png)

To know more about point-to-point communication click [here](https://www.sciencedirect.com/topics/computer-science/point-to-point-communication)

**Passing an Integer**

In [1]:
from mpi4py import MPI

ModuleNotFoundError: No module named 'mpi4py'

In [15]:
%%writefile comm.py
from mpi4py import MPI # Importing mpi4py package from MPI module
# Defining a function
def main():
    # Creating a Communicator
    comm = MPI.COMM_WORLD
    #number of the process running the code
    rank = comm.Get_rank()
    # total number of processes running
    size = comm.Get_size()
    # master process
    if rank == 0:
        data = 123 # Defining a integer
        # master process sends data to worker processes by
        # going through the ranks of all worker processes
        for i in range(1, size):
            # Sending the data to each process
            comm.send(data, dest=i, tag=i)
            print('Process {} sent data:'.format(rank), data)
    # worker processes
    else:
        # each worker process receives data from master process
        data = comm.recv(source=0, tag=rank)
        print('Process {} received data:'.format(rank), data)
main()

Writing comm.py


Now let us see how we can use the **mpirun** program to execute the above python code using 2 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [16]:
!mpirun --allow-run-as-root --oversubscribe -np 2 python comm.py

Process 0 sent data: 123
Process 1 received data: 123


#### Exercise

- Run the above code, using different number of processes.

**Passing a Python Dictionary**

In [17]:
%%writefile passing_dict.py
from mpi4py import MPI # Importing mpi4py package from MPI module
# Defining a function
def main():
    # Creating a communicator
    comm = MPI.COMM_WORLD
    # number of the process running the code
    rank = comm.Get_rank()
    # total number of processes running
    size = comm.Get_size()
    # master process
    if rank == 0:
        # Generate a dictionary with arbitrary data in it
        data = {'States' : ["Hyderabad", "Goa", "Punjab"]}
        # master process sends data to worker processes by
        # going through the ranks of all worker processes
        for i in range(1, size):
            # Sending data
            comm.send(data, dest=i, tag=i)
            # Displaying the results
            print('Process {} sent data:'.format(rank), data)
    # worker processes
    else:
        # each worker process receives data from master process
        data = comm.recv(source=0, tag=rank)
        # Displaying the results
        print('Process {} received data:'.format(rank), data)
# Calling the function
main()

Writing passing_dict.py


Now let us see how we can use the **mpirun** program to execute the above python code using 3 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [18]:
!mpirun --allow-run-as-root --oversubscribe -np 3 python passing_dict.py

Process 0 sent data: {'States': ['Hyderabad', 'Goa', 'Punjab']}
Process 0 sent data: {'States': ['Hyderabad', 'Goa', 'Punjab']}
Process 2 received data: {'States': ['Hyderabad', 'Goa', 'Punjab']}
Process 1 received data: {'States': ['Hyderabad', 'Goa', 'Punjab']}


#### Exercise

- Run the above code, using different number of processes.

#### Research Question

- Write a code to send and receive a Tuple, List, NumPy array, and User Input from one process to another process.

###  Non-blocking communication

So far we have seen how to send and receive messages using blocking communication. In this case, the sender or receiver is not able to perform any other actions until the corresponding message has been sent or received.

Blocking communication has a number of disadvantages. Potential computational time is simply wasted while waiting for the call to complete. An alternate approach is to allow the program to continue execution while the messages are being sent or received. This is known as **non-blocking communication.**

To know more about Non-blocking communication click [here](https://www.mpi-forum.org/docs/mpi-1.1/mpi-11-html/node44.html#:~:text=A%20nonblocking%20send%20start%20call,out%20of%20the%20send%20buffer.)

Now let us create a non-blocking version of the send and receive program. Note there is no need to wait after process 1 sends the message, nor after process 0 sends the reply. However it is necessary for process 1 to wait for the reply so that it knows the message has been fully received before trying to print it out. Similarly, process 0 must wait for the full message before trying to compute randNum * 2.

In [19]:
%%writefile nonblockingarray.py
from mpi4py import MPI # Importing mpi4py package from MPI module
import numpy as np # Importing Numpy Package undername np
# Defining a function
def main():
    # Creating a communicator
    comm = MPI.COMM_WORLD
    # number of the process running the code
    rank = comm.Get_rank()
    # Generating a random number
    randNum = np.zeros(1)
    # Generating a random float value in the open interval
    diffNum = np.random.random_sample(1)
    if rank == 1:
        # Generating a random float value in the open interval
        randNum = np.random.random_sample(1)
        # Display the results
        print("Process", rank, "drew the number", randNum[0])
        # Sending the random number to the processes
        comm.Isend(randNum, dest=0)
        # Receiving the Passed random number from the process
        req = comm.Irecv(randNum, source=0)
        # Waiting for the reply
        req.Wait()
        # Displaying the received random number
        print("Process", rank, "received the number", randNum[0])
    if rank == 0:
        # Displaying the results before receiving the number
        print("Process", rank, "before receiving has the number", randNum[0])
        # Receiving the randomNumber
        req = comm.Irecv(randNum, source=1)
        # Waiting for the reply
        req.Wait()
        # Displaying the results
        print("Process", rank, "received the number", randNum[0])
        randNum *= 2
        # Sending the random Number
        comm.Isend(randNum, dest=1)
# Calling the function
main()

Writing nonblockingarray.py


Now let us see how we can use the **mpirun** program to execute the above python code using 4 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [20]:
!mpirun --allow-run-as-root --oversubscribe -np 4 python nonblockingarray.py

Process 1 drew the number 0.557272667426449
Process 0 before receiving has the number 0.0
Process 0 received the number 0.557272667426449
Process 1 received the number 1.114545334852898


#### Exercise

- Run the above code, using different number of processes.

**Overlapping communication and computation**

Now let us modify the above code so that process 1 overlaps a computation with sending the message and receiving the reply. The computation is to divide diffNum by 3.14 and print the result.

In [21]:
%%writefile overlappingCommunication.py
from mpi4py import MPI # Importing mpi4py package from MPI module
import numpy as np # Importing Numpy Package undername np
# Defining a function
def main():
    # Creating a communicator
    comm = MPI.COMM_WORLD
    # number of the process running the code
    rank = comm.Get_rank()
    # Generating a random number
    randNum = np.zeros(1)
    # Generating a random float value in the open interval
    diffNum = np.random.random_sample(1)
    if rank == 1:
        # Generating a random float value in the open interval
        randNum = np.random.random_sample(1)
        # Display the results
        print("Process", rank, "drew the number", randNum[0])
        # Sending the random number to the processes
        comm.Isend(randNum, dest=0)
        diffNum /= 3.14 # overlap communication
        # Displaying the result
        print("diffNum=", diffNum[0])
        # Receiving the Passed random number from the process
        req = comm.Irecv(randNum, source=0)
        # Waiting for the reply
        req.Wait()
        # Displaying the received random number
        print("Process", rank, "received the number", randNum[0])
    if rank == 0:
        # Displaying the results before receiving the number
        print("Process", rank, "before receiving has the number", randNum[0])
        # Receiving the randomNumber
        req = comm.Irecv(randNum, source=1)
        # Waiting for the reply
        req.Wait()
        # Displaying the results
        print("Process", rank, "received the number", randNum[0])
        randNum *= 2
        # Sending the random Number
        comm.Isend(randNum, dest=1)
# Calling the function
main()

Writing overlappingCommunication.py


Now let us see how we can use the **mpirun** program to execute the above python code using 2 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [22]:
!mpirun --allow-run-as-root --oversubscribe -np 2 python overlappingCommunication.py

Process 1 drew the number 0.414254485450827
diffNum= 0.2977514580308489
Process 0 before receiving has the number 0.0
Process 0 received the number 0.414254485450827
Process 1 received the number 0.828508970901654


#### Exercise

- Run the above code, using different number of processes.

### Collective Communication

In addition to point-to-point communications between individual pairs of processors, MPI includes routines for performing collective communications. These routines allow larger groups of processors to communicate in various ways.

Examples of collective communications include broadcast operations, gather and scatter operations, and reduction operations. Now let us try to understand them one by one.

To know more about collective communication click [here](https://www.mpi-forum.org/docs/mpi-1.1/mpi-11-html/node64.html#:~:text=Collective%20communication%20is%20defined%20as,members%20of%20a%20group%20(Sec.)

#### 1. Broadcast

The simplest kind of collective operation is the broadcast. In a broadcast operation a single process sends a copy of some data to all the other processes in a group.

The communication pattern of a broadcast looks like this:

![Image](https://cdn.iisc.talentsprint.com/CDS/Images/broadcast.png)

In the above figure, process zero is the root process, and it has the initial copy of data. All of the other processes receive the copy of data.

**Broadcasting a Python Dictionary**

In [24]:
! mpirun --allow-run-as-root --oversubscribe -np 4 python BroadcastingDictionary.py

Rank:  0 , data:  {'States': ['Hyderabad', 'Goa', 'Punjab']}
Rank:  2 , data:  {'States': ['Hyderabad', 'Goa', 'Punjab']}
Rank:  1 , data:  {'States': ['Hyderabad', 'Goa', 'Punjab']}
Rank:  3 , data:  {'States': ['Hyderabad', 'Goa', 'Punjab']}


#### Exercise

- Run the above code, using different number of processes.

**Broadcasting a NumPy array**

In [25]:
%%writefile BroadcastingNumpy.py
from mpi4py import MPI # Importing mpi4py package from MPI module
import numpy as np # Importing numpy package under a name np
# Defining a function
def main():
    # communicator
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()   # number of the process running the code
    size = comm.Get_size()   # total number of processes running
    if rank == 0:
        # Generate a Numpy arary with arbitrary data in it
        data = np.ones(4)
    else:
        # start with empty data
        data = None
    # Broadcasting the data
    data = comm.bcast(data, root=0)
    # Printing the results
    if rank == 0:
        print('Process {} broadcast data:'.format(rank), data)
    else:
        print('Process {} received data:'.format(rank), data)
# Calling the main function
main()

Writing BroadcastingNumpy.py


Now let us see how we can use the **mpirun** program to execute the above python code using 3 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [26]:
! mpirun --allow-run-as-root --oversubscribe -np 4 python BroadcastingNumpy.py

Process 0 broadcast data: [1. 1. 1. 1.]
Process 2 received data: [1. 1. 1. 1.]
Process 1 received data: [1. 1. 1. 1.]
Process 3 received data: [1. 1. 1. 1.]


#### Research Question

- Write a code to Broadcast a Integer, List, and User Input

#### 2. Gather and Scatter Operations

**Scatter Operation**

Scatter takes an array and distributes contiguous sections of it across the ranks of a communicator. The communication pattern of a Scatter looks like this:

![Image](https://cdn.iisc.talentsprint.com/CDS/Images/Scatter.png)

Broadcast takes a single data element at the root process (the red box) and copies it to all other processes. However the scatter takes an array of elements and distributes the elements in the order of process rank. The first element (in red) goes to process zero, the second element (in green) goes to process one, and so on. Although the root process (process zero) contains the entire array of data, the scatter operation will copy the appropriate element into the receiving buffer of the process.

**Gather Operation**

The reverse of a scatter is a gather, which takes subsets of an array that are distributed across the ranks, and gathers them back into the full array. The communication pattern of a gather looks like this:

![Image](https://cdn.iisc.talentsprint.com/CDS/Images/gather.png)

Similar to scatter, gather takes elements from each process and gathers them to the root process. The elements are ordered by the rank of the process from which they were received.

To know more about Gather and Scatter Operation click [here](https://www.codingame.com/playgrounds/349/introduction-to-mpi/scattering-and-gathering)

**Scatter Operation on a NumPy Array**

In [27]:
%%writefile ScatteringNumpyArray.py
from mpi4py import MPI # Importing mpi4py package from MPI module
import numpy as np # Importing numpy package under a name np
# Defining a function
def main():
    # communicator
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()   # number of the process running the code
    size = comm.Get_size()   # total number of processes running
    numDataPerRank = 10   # Number of elements in a array for each rank
    data = None # Starting with an empty  data
    if rank == 0:
        # Creating a Numpy array.
        data = np.linspace(1, size * numDataPerRank,numDataPerRank * size)
    # when size = 2 (using -n 2), data = [1.0:20.0]
    recvbuf = np.empty(numDataPerRank, dtype='d') # allocate space for recvbuf
    # scatter operation
    comm.Scatter(data, recvbuf, root=0)
    # Displaying the result
    print('Rank: ',rank, ', recvbuf received: ',recvbuf)
# Calling the main function
main()

Writing ScatteringNumpyArray.py


Now let us see how we can use the **mpirun** program to execute the above python code using 6 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [28]:
! mpirun --allow-run-as-root --oversubscribe -np 6 python ScatteringNumpyArray.py

Rank:  0 , recvbuf received:  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
Rank:  4 , recvbuf received:  [41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
Rank:  1 , recvbuf received:  [11. 12. 13. 14. 15. 16. 17. 18. 19. 20.]
Rank:  2 , recvbuf received:  [21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]
Rank:  5 , recvbuf received:  [51. 52. 53. 54. 55. 56. 57. 58. 59. 60.]
Rank:  3 , recvbuf received:  [31. 32. 33. 34. 35. 36. 37. 38. 39. 40.]


#### Exercise

- Run the above code, using different number of processes.

**Gather Operation on a NumPy Array**

In [29]:
%%writefile GatherringNumPyArray.py
from mpi4py import MPI # Importing mpi4py package from MPI module
import numpy as np # Importing numpy package under a name np
# Defining a function
def main():
    # communicator
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()   # number of the process running the code
    size = comm.Get_size()   # total number of processes running
    numDataPerRank = 10   # Number of elements in a array for each rank
    # Creating a sender buffer array
    sendbuf = np.linspace(rank * numDataPerRank + 1,(rank + 1) * numDataPerRank,numDataPerRank)
    # Printing the result
    print('Rank: ',rank, ', sendbuf: ',sendbuf)
    recvbuf = None
    if rank == 0:
        # Creating a receiver buffer array
        recvbuf = np.empty(numDataPerRank * size, dtype='d')
    # Gathering the Information
    comm.Gather(sendbuf, recvbuf, root = 0)
    # Display the result
    if rank == 0:
        print('Rank: ',rank, ', recvbuf received: ',recvbuf)
# Calling a function
main()

Writing GatherringNumPyArray.py


Now let us see how we can use the **mpirun** program to execute the above python code using 2 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [30]:
! mpirun --allow-run-as-root --oversubscribe -np 2 python GatherringNumPyArray.py

Rank:  0 , sendbuf:  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
Rank:  1 , sendbuf:  [11. 12. 13. 14. 15. 16. 17. 18. 19. 20.]
Rank:  0 , recvbuf received:  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20.]


#### Exercise

- Run the above code, using different number of processes.

#### Research Question

- Write a code to perform Gather and Scatter operation on a Python List

#### 3. Reduction Operation

Reduce is a classic concept from functional programming. Data reduction involves reducing a set of numbers into a smaller set of numbers via a function. For example, let’s say we have a list of numbers [1, 2, 3, 4, 5]. Reducing this list of numbers with the sum function would produce sum([1, 2, 3, 4, 5]) = 15. Similarly, the multiplication reduction would yield multiply([1, 2, 3, 4, 5]) = 120.

The communication pattern of a reduction looks like this:

![Image](https://cdn.iisc.talentsprint.com/CDS/Images/reduction.png)

In the above image, each process contains one integer. The reduction operation is called with a root process of 0 and using MPI_SUM as the reduction operation. The four numbers are summed to the result and stored on the root process.

It is also useful to see what happens when processes contain multiple elements. The illustration below shows reduction of multiple numbers per process.

![Image](https://cdn.iisc.talentsprint.com/CDS/Images/reduction1.png)

To know more about reduction operation click [here](https://www.codingame.com/playgrounds/349/introduction-to-mpi/reductions)

**Reduction Operation on all values of Array using sum and max**

In [31]:
%%writefile ReducingNumpyArray.py
from mpi4py import MPI # Importing mpi4py package from MPI module
import numpy as np # Importing numpy package under a name np
# Defining a function
def main():
    # communicator
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()   # number of the process running the code
    size = comm.Get_size()   # total number of processes running
    # Create numpy arrays on each process: For this experiment, the arrays have only one
    # entry that is assigned to be the rank of the processor
    value = np.array(rank,'d')
    # Displaying the value and its rank
    print(' Rank: ',rank, ' value = ', value)
    # initialize the np arrays that will store the results:
    value_sum      = np.array(0.0,'d')
    value_max      = np.array(0.0,'d')
    # perform the reductions:
    comm.Reduce(value, value_sum, op=MPI.SUM, root=0)
    comm.Reduce(value, value_max, op=MPI.MAX, root=0)
    # Displaying the result
    if rank == 0:
        print(' Rank 0: value_sum =    ',value_sum)
        print(' Rank 0: value_max =    ',value_max)
# Calling a function
main()

Writing ReducingNumpyArray.py


Now let us see how we can use the **mpirun** program to execute the above python code using 5 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [32]:
! mpirun --allow-run-as-root --oversubscribe -np 2 python ReducingNumpyArray.py

 Rank:  0  value =  0.0
 Rank:  1  value =  1.0
 Rank 0: value_sum =     1.0
 Rank 0: value_max =     1.0


#### Exercise

- Run the above code, using different number of processes.

#### Research Question

- Try replacing MPI.MAX with MPI.MIN(minimum) and/or replacing MPI.SUM with MPI.PROD (product). Then save and run the code again.
- Write a code to perform reduction operation on values of a list

### Please answer the questions below to complete the experiment:




In [33]:
# @title Mark the following statement as TRUE or FALSE: Scatter sends the same piece of data to all processes while Broadcast sends sections of an array to different processes { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "FALSE" #@param ["","TRUE","FALSE"]

In [34]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Was Tough, but I did it" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [35]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "Not easy - I need to practice the research questions and code them too. " #@param {type:"string"}


In [36]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]


In [37]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Somewhat Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [38]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "Didn't use" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [39]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 4660
Date of submission:  03 May 2025
Time of submission:  15:27:34
View your submissions: https://learn-iisc.talentsprint.com/notebook_submissions
